In [1]:
import numpy as np
from datasets import load_dataset
from datasets import Dataset, DatasetDict, IterableDataset, IterableDatasetDict
import torch
import torch.nn as nn

In [2]:
train_datasets = load_dataset("json", data_files=["./data/train.json"])
test_datasets = load_dataset("json", data_files=["./data/val.json"])

Using custom data configuration default-b11d075f0280a41a
Reusing dataset json (C:\Users\pc\.cache\huggingface\datasets\json\default-b11d075f0280a41a\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-e0ddbe2a7a74f79e
Reusing dataset json (C:\Users\pc\.cache\huggingface\datasets\json\default-e0ddbe2a7a74f79e\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
for p in iter(train_datasets['train']):
    print(p)
    break

{'tokens': ['浙', '江', '杭', '州', '市', '江', '干', '区', '九', '堡', '镇', '三', '村', '村', '一', '区'], 'ner_tags': ['B-prov', 'E-prov', 'B-city', 'I-city', 'E-city', 'B-district', 'I-district', 'E-district', 'B-town', 'I-town', 'E-town', 'B-community', 'I-community', 'E-community', 'B-poi', 'E-poi']}


In [4]:
test_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1970
    })
})

In [5]:
from transformers import BertTokenizer,DataCollatorWithPadding, TrainingArguments
from transformers import BertForTokenClassification,BertModel
from transformers import Trainer
import torch
import numpy as np

In [6]:
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [7]:
# model = BertForTokenClassification.from_pretrained(model_name)

In [8]:
# # 假设训练数据集包含以下数据：
# data = [("这是一个例子句子。", ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']),
#         ("我来自北京市海淀区。", ['O', 'O', 'B-city', 'B-district', 'O', 'O', 'O', 'O']),
#         ("我喜欢看《红楼梦》。", ['O', 'O', 'O', 'O', 'O', 'O', 'B-poi', 'E-poi'])]

# # 将标签转换为对应的ID
# label2id = {'O': 0, 'B-city': 1, 'B-district': 2, 'B-poi': 3, 'E-poi': 4}
# train_data = []
# for text, labels in data:
#     label_ids = [label2id[label] for label in labels]
#     train_data.append((text, label_ids))
# train_data


# input_ids = torch.tensor(tokenizer.encode("哈喽，你好")).unsqueeze(0)  # Batch size 1
# labels = torch.tensor([1] * input_ids.size(1)).unsqueeze(0)  # Batch size 1
# print(input_ids,labels)
# outputs = model(input_ids, labels=labels)
# loss, scores = outputs[:2]

In [9]:
# # trainloader = torch.utils.data.DataLoader(train_datasets['train'],batch_size=64,shuffle=True,num_workers=2)
# # testloader = torch.utils.data.DataLoader(testset,batch_size=64,shuffle=False,num_workers=2)
# import torch
# import torch.utils.data as Data
# class MyDataSet(Data.Dataset):
#     def __init__(self, data):
#         super(MyDataSet, self).__init__()
#         self.tokens = data['tokens']
#         self.labels = data['ner_tags']
#     def __len__(self):
#         return len(self.tokens)
#     def __getitem__(self,idx):
#         tokenizer.encode(self.tokens)
#         return tokenizer.encode(self.tokens), [label2id[0][n] for n in self.labels]

# def label2id_func(x):
#     return label2id[0][x]
# #atasetes = MyDataSet(train_datasets['train'])
# encode_data = []
# for i, data in enumerate(train_datasets['train']):
#     tokens = data['tokens']
#     ner_tags = data['ner_tags']
    
#     res = [tokenizer.encode(data["tokens"]),[label2id[0][n] for n in data['ner_tags']] ]
    
#     print(tokens,ner_tags,res)
#     break

# list(map(label2id_func,data['ner_tags']))


In [10]:
# trainloader = torch.utils.data.DataLoader(train_datasets['train'],batch_size=64,shuffle=True,num_workers=2)
# trainloader

In [11]:
label2id={
            "B-assist": 0,
            "B-cellno": 1,
            "B-city": 2,
            "B-community": 3,
            "B-devzone": 4,
            "B-distance": 5,
            "B-district": 6,
            "B-floorno": 7,
            "B-houseno": 8,
            "B-intersection": 9,
            "B-poi": 10,
            "B-prov": 11,
            "B-road": 12,
            "B-roadno": 13,
            "B-subpoi": 14,
            "B-town": 15,
            "B-village_group": 16,
            "E-assist": 17,
            "E-cellno": 18,
            "E-city": 19,
            "E-community": 20,
            "E-devzone": 21,
            "E-distance": 22,
            "E-district": 23,
            "E-floorno": 24,
            "E-houseno": 25,
            "E-intersection": 26,
            "E-poi": 27,
            "E-prov": 28,
            "E-road": 29,
            "E-roadno": 30,
            "E-subpoi": 31,
            "E-town": 32,
            "E-village_group": 33,
            "I-assist": 34,
            "I-cellno": 35,
            "I-city": 36,
            "I-community": 37,
            "I-devzone": 38,
            "I-distance": 39,
            "I-district": 40,
            "I-floorno": 41,
            "I-houseno": 42,
            "I-intersection": 43,
            "I-poi": 44,
            "I-prov": 45,
            "I-road": 46,
            "I-roadno": 47,
            "I-subpoi": 48,
            "I-town": 49,
            "I-village_group": 50,
            "O": 51,
            "S-assist": 52,
            "S-community": 53,
            "S-district": 54,
            "S-intersection": 55,
            "S-poi": 56
        },
num_labels= 57
id2label={
            "0": "B-assist",
            "1": "B-cellno",
            "2": "B-city",
            "3": "B-community",
            "4": "B-devzone",
            "5": "B-distance",
            "6": "B-district",
            "7": "B-floorno",
            "8": "B-houseno",
            "9": "B-intersection",
            "10": "B-poi",
            "11": "B-prov",
            "12": "B-road",
            "13": "B-roadno",
            "14": "B-subpoi",
            "15": "B-town",
            "16": "B-village_group",
            "17": "E-assist",
            "18": "E-cellno",
            "19": "E-city",
            "20": "E-community",
            "21": "E-devzone",
            "22": "E-distance",
            "23": "E-district",
            "24": "E-floorno",
            "25": "E-houseno",
            "26": "E-intersection",
            "27": "E-poi",
            "28": "E-prov",
            "29": "E-road",
            "30": "E-roadno",
            "31": "E-subpoi",
            "32": "E-town",
            "33": "E-village_group",
            "34": "I-assist",
            "35": "I-cellno",
            "36": "I-city",
            "37": "I-community",
            "38": "I-devzone",
            "39": "I-distance",
            "40": "I-district",
            "41": "I-floorno",
            "42": "I-houseno",
            "43": "I-intersection",
            "44": "I-poi",
            "45": "I-prov",
            "46": "I-road",
            "47": "I-roadno",
            "48": "I-subpoi",
            "49": "I-town",
            "50": "I-village_group",
            "51": "O",
            "52": "S-assist",
            "53": "S-community",
            "54": "S-district",
            "55": "S-intersection",
            "56": "S-poi"
        }

In [12]:
tokenized_datasets = train_datasets['train']
tokenized_datasets

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 8856
})

In [13]:
# 准备训练数据
train_encodings = tokenizer([''.join(data["tokens"]) for data in iter(tokenized_datasets)], truncation=True, padding=True)
max_len = len(train_encodings['input_ids'][0])
print(max_len)
train_labels = {"labels": [ [label2id[0]['O']] + [label2id[0][tag] for tag in data["ner_tags"]] 
                 + [label2id[0]['O']]*(max_len-len(data["ner_tags"])-1) if len(data["ner_tags"]) <= max_len-2 else 
                 [label2id[0]['O']] + [label2id[0][tag] for tag in data["ner_tags"][:max_len-2]] + [label2id[0]['O']]
                  for data in iter(tokenized_datasets)]}#encode后会加S和E以及padding，label要保证长度一致

print(np.array(train_labels['labels']).shape, np.array(train_encodings['input_ids']).shape)



67
(8856, 67) (8856, 67)


In [14]:
from torch.utils.data import TensorDataset, DataLoader
train_encodings['labels'] = train_labels['labels']

# #attention_mask = attention_mask#torch.tensor([[1] * (len(tokenized_text[:max_len])+2) + [0] * (max_len - len(tokenized_text[:max_len])) for tokenized_text in iter(tokenized_datasets)])
# print(np.array(train_encodings.attention_mask).shape)
# from torch.utils.data import TensorDataset, DataLoader
# # 将输入数据转换为TensorDataset类型，用于训练和评估
dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), torch.tensor(train_labels['labels'] ))


In [15]:
# 将数据加载到DataLoader中，用于批量训练和评估
BATCH_SIZE = 64
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
dataset

In [16]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 初始化BERT模型
model = BertForTokenClassification.from_pretrained('bert-base-chinese',num_labels = len(label2id[0]))
# model = nn.DataParallel(model)#多卡训练
# model = model.to(device)


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-c

In [17]:
model = model.to(device)
# 定义优化器和学习率
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
running_loss = 0
# 训练模型
EPOCHS =5
for epoch in range(EPOCHS):
    print('Epoch:', epoch + 1)
    for i,batch in enumerate(dataloader):
        
        batch_input_ids = batch[0].to(device)
        batch_attention_mask = batch[1].to(device)
        batch_labels = batch[2].to(device)

        # 计算模型的损失
        outputs = model(batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
        loss = outputs[0]#.mean(dim=0)# 多GPU训练需要加.mean(dim=0)，否则直接使用outputs即可

        # 反向传播和优化
        loss.backward()
      
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.item()
     
        if i%10 == 9:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 10.0))
            running_loss = 0.0


Epoch: 1
[1,    10] loss: 1.63724
[1,    20] loss: 0.72477
[1,    30] loss: 0.45325
[1,    40] loss: 0.31002
[1,    50] loss: 0.24236
[1,    60] loss: 0.19151
[1,    70] loss: 0.16512
[1,    80] loss: 0.15471
[1,    90] loss: 0.16183
[1,   100] loss: 0.13879
[1,   110] loss: 0.12499
[1,   120] loss: 0.10749
[1,   130] loss: 0.12143
Epoch: 2
[2,    10] loss: 0.19020
[2,    20] loss: 0.09459
[2,    30] loss: 0.09590
[2,    40] loss: 0.08866
[2,    50] loss: 0.09184
[2,    60] loss: 0.07984
[2,    70] loss: 0.08518
[2,    80] loss: 0.08101
[2,    90] loss: 0.09260
[2,   100] loss: 0.07934
[2,   110] loss: 0.06723
[2,   120] loss: 0.07133
[2,   130] loss: 0.07869
Epoch: 3
[3,    10] loss: 0.12715
[3,    20] loss: 0.06303
[3,    30] loss: 0.05643
[3,    40] loss: 0.05212
[3,    50] loss: 0.05586
[3,    60] loss: 0.05877
[3,    70] loss: 0.05257
[3,    80] loss: 0.05080
[3,    90] loss: 0.04876
[3,   100] loss: 0.05420
[3,   110] loss: 0.05298
[3,   120] loss: 0.05332
[3,   130] loss: 0.0573

In [18]:
torch.cuda.is_available()

True

In [19]:
import torch

# 获取GPU设备数量
num_devices = torch.cuda.device_count()
print("当前计算机有{}个GPU设备".format(num_devices))

# 获取每个GPU设备的名称
for i in range(num_devices):
    device_name = torch.cuda.get_device_name(i)
    print("设备{}的名称为：{}".format(i, device_name))


当前计算机有2个GPU设备
设备0的名称为：Quadro RTX 4000
设备1的名称为：Quadro RTX 4000


In [20]:
with torch.no_grad():
    for i,batch in enumerate(dataloader):
        batch_input_ids = batch[0].to(device)
        batch_attention_mask = batch[1].to(device)
        batch_labels = batch[2].to(device)

        # 计算模型的损失
        outputs = model(batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
        loss = outputs[0].mean(dim=0)# 多GPU训练需要加.mean(dim=0)，否则直接使用outputs即可
        break
print(torch.max(outputs[1],2))

torch.return_types.max(
values=tensor([[10.9694,  9.8041,  9.8926,  ..., 12.4295, 12.3193, 12.3640],
        [11.3796, 10.7151, 10.7688,  ..., 12.5885, 12.5957, 12.5869],
        [11.2269, 11.0188, 10.8809,  ..., 12.5843, 12.5967, 12.5834],
        ...,
        [11.2177, 10.6192, 11.0087,  ..., 12.5550, 12.5434, 12.5630],
        [11.3084, 10.8719, 10.4875,  ..., 12.4627, 12.4483, 12.4550],
        [10.9361, 10.9430, 10.5469,  ..., 12.4301, 12.4269, 12.4429]],
       device='cuda:0'),
indices=tensor([[51, 10, 44,  ..., 51, 51, 51],
        [51,  2, 19,  ..., 51, 51, 51],
        [51,  2, 36,  ..., 51, 51, 51],
        ...,
        [51,  6, 40,  ..., 51, 51, 51],
        [51, 11, 45,  ..., 51, 51, 51],
        [51, 11, 45,  ..., 51, 51, 51]], device='cuda:0'))


In [21]:
batch_labels.shape

torch.Size([64, 67])

In [22]:
outputs[1].shape#torch.max(outputs[1],1)


torch.Size([64, 67, 57])

In [23]:
_,pre = torch.max(outputs[1],2)
pre.shape

torch.Size([64, 67])

In [24]:
pre

tensor([[51, 10, 44,  ..., 51, 51, 51],
        [51,  2, 19,  ..., 51, 51, 51],
        [51,  2, 36,  ..., 51, 51, 51],
        ...,
        [51,  6, 40,  ..., 51, 51, 51],
        [51, 11, 45,  ..., 51, 51, 51],
        [51, 11, 45,  ..., 51, 51, 51]], device='cuda:0')

In [25]:
batch_labels

tensor([[51, 10, 44,  ..., 51, 51, 51],
        [51,  2, 19,  ..., 51, 51, 51],
        [51,  2, 36,  ..., 51, 51, 51],
        ...,
        [51,  6, 40,  ..., 51, 51, 51],
        [51, 11, 45,  ..., 51, 51, 51],
        [51, 11, 45,  ..., 51, 51, 51]], device='cuda:0')

In [26]:
val_datasets = test_datasets['train']
val_datasets

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 1970
})

In [27]:
# 准备训练数据
test_encodings = tokenizer([''.join(data["tokens"]) for data in iter(val_datasets)], truncation=True, padding=True)
max_len = len(test_encodings['input_ids'][0])
print(max_len)
test_labels = {"labels": [ [label2id[0]['O']] + [label2id[0][tag] for tag in data["ner_tags"]] 
                 + [label2id[0]['O']]*(max_len-len(data["ner_tags"])-1) if len(data["ner_tags"]) <= max_len-2 else 
                 [label2id[0]['O']] + [label2id[0][tag] for tag in data["ner_tags"][:max_len-2]] + [label2id[0]['O']]
                  for data in iter(val_datasets)]}#encode后会加S和E以及padding，label要保证长度一致

print(np.array(test_labels['labels']).shape, np.array(test_encodings['input_ids']).shape)



44
(1970, 44) (1970, 44)


In [28]:
from torch.utils.data import TensorDataset, DataLoader
test_encodings['labels'] = test_labels['labels']

# #attention_mask = attention_mask#torch.tensor([[1] * (len(tokenized_text[:max_len])+2) + [0] * (max_len - len(tokenized_text[:max_len])) for tokenized_text in iter(tokenized_datasets)])
# print(np.array(train_encodings.attention_mask).shape)
# from torch.utils.data import TensorDataset, DataLoader
# # 将输入数据转换为TensorDataset类型，用于训练和评估
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']), torch.tensor(test_encodings['attention_mask']), torch.tensor(test_labels['labels'] ))
# 将数据加载到DataLoader中，用于批量训练和评估
BATCH_SIZE = 64
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataset

In [29]:
total_loss = 0
test_labels = []
res = []
with torch.no_grad():
    for i,batch in enumerate(testloader):
        batch_input_ids = batch[0].to(device)
        batch_attention_mask = batch[1].to(device)
        batch_labels = batch[2].to(device)

        # 计算模型的损失
        outputs = model(batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
        loss = outputs[0].mean(dim=0)# 多GPU训练需要加.mean(dim=0)，否则直接使用outputs即可
        total_loss += loss.item()
        _,tmp_res = torch.max(outputs[1],2)
        res.append(tmp_res.cpu().numpy())
        test_labels.append(batch_labels.cpu().numpy())

    total_loss = total_loss/(BATCH_SIZE*i)
print(np.concatenate(res,axis=0).shape)

(1970, 44)


In [30]:
np.concatenate(res,axis=0)

array([[51, 15, 44, ..., 51, 51, 51],
       [51, 11, 45, ..., 51, 51, 51],
       [51,  6, 23, ..., 51, 51, 51],
       ...,
       [51, 12, 46, ..., 51, 51, 51],
       [51, 12, 46, ..., 51, 51, 51],
       [51,  6, 40, ..., 51, 51, 51]], dtype=int64)

In [31]:
np.concatenate(test_labels,axis=0)


array([[51,  6, 23, ..., 51, 51, 51],
       [51, 11, 45, ..., 51, 51, 51],
       [51, 15, 32, ..., 51, 51, 51],
       ...,
       [51, 12, 46, ..., 51, 51, 51],
       [51, 12, 46, ..., 51, 51, 51],
       [51,  6, 40, ..., 51, 51, 51]], dtype=int64)

In [32]:
total_loss

0.002326412700737516

In [33]:
correct =(np.concatenate(res,axis=0) == np.concatenate(test_labels,axis=0)).sum().item()/(1970*44)

In [34]:
len_matric = []
for data in iter(test_datasets['train']):
    len_matric.append(len(data['tokens']))
print(len(len_matric))

1970


In [35]:

correct

0.9657821873557915

In [36]:
test_res_src = [ data[1: min(len_matric[i], max_len-2)]
                  for i,data in enumerate(np.concatenate(res,axis=0))]#encode后会加S和E以及padding，label要保证长度一致

In [37]:
test_label_src = [ data[1: min(len_matric[i], max_len-2)]
                  for i,data in enumerate(np.concatenate(test_labels,axis=0))]#encode后会加S和E以及padding，label要保证长度一致

In [38]:
correct =(np.concatenate(test_res_src,axis=0) == np.concatenate(test_label_src,axis=0)).sum().item()/sum(len_matric)
correct

0.8799083868245788

loss: 0.05307
correct: 0.876
epoch: 3
opt: Adam
lr: 5e-5

# 使用tranformer库中的Trainer函数实现微调
- 注意，前面使用的时Tensordatasets, 在使用Trainer时我们需要传输如Dataset类，需要自定义个类转化一下
> Dataset是PyTorch中的一个抽象类，用于表示数据集。它定义了一个标准的接口，使得我们可以对各种不同格式的数据进行统一的处理。Dataset类本身不存储任何数据，而是提供了一种统一的方法来访问数据。

> TensorDataset是Dataset类的一种实现，它接受多个张量作为输入，每个张量的第一个维度应该相同，用于表示样本数。TensorDataset将这些张量打包成一个元组，并将每个元组作为一个样本。因此，它适用于对张量数据进行训练和测试。

> Dataset和TensorDataset的区别在于它们所接受的数据类型不同。Dataset可以接受各种不同类型的数据，例如文本、图像、音频等，而TensorDataset只能接受张量数据。此外，Dataset通常需要实现__getitem__和__len__方法，以便可以对数据进行索引和获取数据集的长度，而TensorDataset已经默认实现了这些方法。

> 给定这个数据集描述 Dataset({ features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], num_rows: 3668 })，如果使用TensorDataset类来表示该数据集，需要先将这些特征和标签数据都转换为张量格式。如果使用Dataset类来表示该数据集，则可以保留原始数据的格式，并将其放入一个列表中，每个元素对应一个样本。在需要访问数据时，使用相应的数据处理方法将数据转换为模型可以处理的格式。

In [39]:
from torch.utils.data import Dataset
from datasets import Dataset, DatasetDict
# class MyDataSet(Dataset):
#     def __init__(self, datasets):
#         super(MyDataSet,self).__init__()
#         self.data = []
#         for i, tmp_data in enumerate(iter(datasets)):
#             item = {}
#             item['input_ids'] = tmp_data[0]
#             item['attention_mask'] = tmp_data[1]
#             item['labels'] = tmp_data[2]
#             self.data.append(item)
#     def __len__(self):
#         return dataset.__len__()
#     def __getitem__(self, index):
#         item = self.data[index]
#         return item
# MyDataSet(dataset).data
dataset_dic = Dataset.from_dict(
    {
        "input_ids": [v[0] for v in iter(dataset)],
        "attention_mask": [v[1] for v in iter(dataset)],
        "labels": [v[2] for v in iter(dataset)],
    }
)

test_dataset_dic = Dataset.from_dict(
    {
        "input_ids": [v[0] for v in iter(test_dataset)],
        "attention_mask": [v[1] for v in iter(test_dataset)],
        "labels": [v[2] for v in iter(test_dataset)],
    }
)


# (可选)将多个Dataset合并为一个DatasetDict
dataset_all = DatasetDict({"train": dataset_dic, "valid": test_dataset_dic})

dataset_all['train'][1].keys()
# for v in iter(dataset):
#     print(v)
#     break
# data = []
# for i, tmp_data in enumerate(iter(dataset)):
#     item = {}
#     item['input_ids'] = tmp_data[0]
#     item['attention_mask'] = tmp_data[1]
#     item['labels'] = tmp_data[2]
#     data.append(item)

#     if i>3:
#         break
# print(data)
#iter(dataset)

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [40]:
dataset_all['valid'].num_rows
# def validate_dataset(dataset):
#     for i, example in enumerate(dataset):
#         if isinstance(example, dict):
#             if "input_ids" not in example:
#                 raise ValueError(f"Example {i} does not contain 'input_ids'.")
#             if "attention_mask" not in example:
#                 raise ValueError(f"Example {i} does not contain 'attention_mask'.")
#             if "labels" not in example:
#                 raise ValueError(f"Example {i} does not contain 'label'.")
#             if len(example["input_ids"]) != len(example["attention_mask"]) or len(example["input_ids"]) != len(example["labels"]):
#                 raise ValueError(f"Example {i} has mismatched input_ids, attention_mask, or label.")
#         elif isinstance(example, tuple):
#             if len(example) != 3:
#                 raise ValueError(f"Example {i} does not have 3 elements.")
#             if not isinstance(example[0], torch.Tensor) or not isinstance(example[1], torch.Tensor) or not isinstance(example[2], torch.Tensor):
#                 raise ValueError(f"Example {i} is not a tuple of tensors.")
#             if example[0].size() != example[1].size() or example[0].size() != example[2].size():
#                 raise ValueError(f"Example {i} has mismatched tensor sizes.")
#         else:
#             raise ValueError(f"Example {i} is not a recognized data type.")
#     print("Dataset validation successful!")
# validate_dataset(dataset_all['valid'])


1970

In [41]:
# #print(dataset,test_dataset)
# 注意，这里不要使用多卡训练！
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model,
    training_args,
    train_dataset = dataset_all["train"],
    eval_dataset = dataset_all["valid"],
    data_collator = data_collator,
    tokenizer = tokenizer,
)

In [42]:
trainer.train()

D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8856
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1662


  0%|          | 0/1662 [00:00<?, ?it/s]

D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
Saving model checkpoint to test-trainer\checkpoint-500
Configuration saved in test-trainer\checkpoint-500\config.json


{'loss': 0.0531, 'learning_rate': 3.495788206979543e-05, 'epoch': 0.9}


Model weights saved in test-trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-500\special_tokens_map.json
D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
***** Running Evaluation *****
  Num examples = 1970
  Batch size = 16


...

Model weights saved in test-trainer\checkpoint-1500\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-1500\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-1500\special_tokens_map.json
D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
***** Running Evaluation *****
  Num examples = 1970
  Batch size = 16


  0%|          | 0/124 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.16120123863220215, 'eval_runtime': 21.0993, 'eval_samples_per_second': 93.368, 'eval_steps_per_second': 5.877, 'epoch': 3.0}
{'train_runtime': 682.3303, 'train_samples_per_second': 38.937, 'train_steps_per_second': 2.436, 'train_loss': 0.036396573740365847, 'epoch': 3.0}


TrainOutput(global_step=1662, training_loss=0.036396573740365847, metrics={'train_runtime': 682.3303, 'train_samples_per_second': 38.937, 'train_steps_per_second': 2.436, 'train_loss': 0.036396573740365847, 'epoch': 3.0})

In [43]:
predictions = trainer.predict(dataset_all["valid"])
print(predictions.predictions.shape) 

***** Running Prediction *****
  Num examples = 1970
  Batch size = 16


  0%|          | 0/124 [00:00<?, ?it/s]

(1970, 44, 57)


In [44]:
print(predictions.label_ids.shape) # array([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...], dtype=int64)
print(predictions.metrics)
pre = np.argmax(predictions.predictions, axis = 2)

(1970, 44)
{'test_loss': 0.16120123863220215, 'test_runtime': 23.3589, 'test_samples_per_second': 84.336, 'test_steps_per_second': 5.308}


In [45]:
predictions.label_ids

array([[51,  2, 19, ..., 51, 51, 51],
       [51, 11, 45, ..., 51, 51, 51],
       [51, 11, 28, ..., 51, 51, 51],
       ...,
       [51,  6, 40, ..., 51, 51, 51],
       [51,  2, 36, ..., 51, 51, 51],
       [51, 11, 45, ..., 51, 51, 51]], dtype=int64)

In [46]:
test_res_src = [ data[1: min(len_matric[i], max_len-2)]
                  for i,data in enumerate(pre)]#encode后会加S和E以及padding，label要保证长度一致

test_label_src = [ data[1: min(len_matric[i], max_len-2)]
                  for i,data in enumerate(predictions.label_ids)]#encode后会加S和E以及padding，label要保证长度一致
correct =(np.concatenate(test_res_src,axis=0) == np.concatenate(test_label_src,axis=0)).sum().item()/sum(len_matric)
correct                 

0.8671608956393334